In [1]:
from brainiak.isc import isc,bootstrap_isc,compute_summary_statistic
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [41]:
froot = '/data00/layerfMRI/'
datafld = froot + 'Github_repo/layerfMRI/analyses/dual_ISC/'
os.listdir(datafld)

['log_summary.csv',
 'fixing_log_file.Rmd',
 '.gitignore',
 'log_summary_FIXED.csv',
 'M_OR_S_JU_time_courses_thr100_bin10.csv',
 'log_summary_OLE_no_Estimated_Duration.csv',
 'dual_ISC_development_v8.html',
 'data_native',
 'ANOVA_M_S_JU_time_courses_thr10_bin10.csv',
 'dual_ISC_development_v8.Rmd',
 'labels_juelich.csv',
 'M_OR_S_JU_time_courses_thr100_bin6.csv',
 'log_summary_FIXED_MRGNCY.csv',
 'ANOVA_M_S_JU_time_courses_thr10_bin6.csv',
 'native_MNI_transformation',
 'OLE',
 'TC_4_lorenzo',
 'fixing_log_file.html']

In [57]:
f = 'M_OR_S_JU_time_courses_thr100_bin6.csv'

In [58]:
logsumm = (pd
           .read_csv(froot+ 'logs/log_summary.csv')
           .assign(fmri = lambda d:
                   froot + 
                   'regdata/sub_' + d['subject'].apply(lambda n: '{:02d}'.format(n)) +
                   '/ses_' + d['session'].apply(lambda n: '{:02d}'.format(n)) +
                   '/func/task_' + d['task'].apply(lambda n: '{:01d}'.format(n)) +
                   '_run_' + d['run'].apply(lambda n: '{:01d}'.format(n)) +
                   '_4D_MNI.nii.gz'
                  )
           .assign(fmri_missing = lambda d : d['fmri'].apply(lambda s: os.path.isfile(s) != True))
           
          )

In [102]:
def r2numpy(rvec):
    
    return np.array(rvec[2:-1].split(', '),dtype = 'float')

# def trim_movielength(dataf):
    
#     trimmed = dataf['tc_mean_unfolded'][:movie_length[dataf['muvi']]]
    
#     return trimmed
    
    
def stdize(a):
    
    return (a - np.average(a)) / np.std(a)



In [103]:
df

,contrast,sub,ses,task,run,muvi,start_TR,end_TR,dim4,ISC_flavour,JU,D_bins,n_bins,n_muvis,n_sub,tc_mean_unfolded
0,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,1,6,36,9,"[-0.0645512051565386, 0.292816308751198, -0.80..."
1,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,2,6,36,9,"[-1.54756027896968, 0.303172837241556, 1.08814..."
2,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,3,6,36,9,"[-1.35370596896645, 0.0406740450402311, 0.3472..."
3,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,4,6,36,9,"[-0.959367812606631, -0.458648927929936, -0.30..."
4,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,5,6,36,9,"[-0.594316313111164, -0.139773280686515, -0.07..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31099,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,2,6,36,9,"[-0.619297310472339, 0.153325600336824, -1.117..."
31100,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,3,6,36,9,"[-0.465676019037052, 1.06732550232737, -1.4665..."
31101,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,4,6,36,9,"[-0.166152853641839, 1.105923662693, -1.315620..."
31102,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,5,6,36,9,"[-0.136988093975467, 1.03780105279179, -1.9854..."


In [107]:
# Read dataframe
df = pd.read_csv(datafld + f)

# Transform timecourses into numpy arrays and standardize them
df['tc_mean_unfolded'] = (df['tc_mean_unfolded']
                          .map(r2numpy)
                         )

df['tc_mean_unfolded']

trimmed = df.apply(lambda row : row['tc_mean_unfolded'][:movie_length[row['muvi']]])


# df['JU'].unique()

KeyError: 'tc_mean_unfolded'

AttributeError: 'DataFrame' object has no attribute 'map'

In [77]:
movie_length = {}

for mv in logsumm.Title.unique():
    durations = logsumm.loc[lambda d : d['Title'] == mv].ExpectedDuration.unique()
    if len(durations) != 1:
        print('Warning! More than one duration found')
        movie_length[mv] = np.min(durations)
    else:
        movie_length[mv] = durations[0]

In [62]:

df

,contrast,sub,ses,task,run,muvi,start_TR,end_TR,dim4,ISC_flavour,JU,D_bins,n_bins,n_muvis,n_sub,tc_mean_unfolded
0,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,1,6,36,9,"[-0.06742153489659666, 0.3058366599180476, -0...."
1,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,2,6,36,9,"[-1.6163739948791278, 0.3166537011386864, 1.13..."
2,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,3,6,36,9,"[-1.4138997715854962, 0.042482654513041264, 0...."
3,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,4,6,36,9,"[-1.0020270001074472, -0.47904318168382426, -0..."
4,Motion,sub_02,ses_01,1,1,M1ballonD.avi,42,54,220,M_OR_S,13,5,6,36,9,"[-0.6207431440957454, -0.14598843040287976, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31099,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,2,6,36,9,"[-0.6338710058304257, 0.15693375517960906, -1...."
31100,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,3,6,36,9,"[-0.47663460116271694, 1.0924424799983548, -1...."
31101,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,4,6,36,9,"[-0.17006286750901228, 1.1319489566460734, -1...."
31102,Scrambled,sub_14,ses_02,4,2,S2boatD.avi,69,91,145,M_OR_S,91,5,6,36,9,"[-0.14021178430243353, 1.062223242473433, -2.0..."


In [61]:
# Get movie names
movie_names = {}
for c,gdf in df.groupby(['contrast']):
    movie_names[c] = gdf.sort_values(['muvi']).muvi.unique()

# Concatenate movie and task
cdf  = pd.DataFrame()
count = 0

for [c,s,j,d],gdf in df.groupby(['contrast','sub','JU','D_bins']):

    gdf = gdf.sort_values(['muvi','run'])
    movie_list = gdf['muvi'].unique()

    try:
        same = (movie_names[c] == movie_list).all()
    except:
        same = False
    if not same:
        print(f'Error! contrast {c} subject {s} JU {j} D_bins {d}')
    else:
        count += 1
    
    concatenated = np.hstack(gdf['tc_mean_unfolded'])
    
    temp_df = gdf.iloc[0][['contrast','sub','task','JU','D_bins']]
    temp_df['tc_concatenated'] = concatenated
    
    cdf = cdf.append(temp_df)

In [38]:
cdf

,D_bins,JU,contrast,sub,task,tc_concatenated
960,1.0,13.0,Motion,sub_02,2.0,"[-0.857982775442284, 0.3961405928276347, -0.81..."
961,2.0,13.0,Motion,sub_02,2.0,"[-0.6121689485606284, -0.41442262115461215, 0...."
962,3.0,13.0,Motion,sub_02,2.0,"[-1.254808798792133, 0.7360159510254902, -0.93..."
963,4.0,13.0,Motion,sub_02,2.0,"[-1.8145595718289669, 0.9421807083579372, -0.3..."
964,5.0,13.0,Motion,sub_02,2.0,"[-1.937179435005998, -0.1234273041502649, 0.93..."
...,...,...,...,...,...,...
28891,2.0,91.0,Scrambled,sub_14,2.0,"[0.09856334081827688, -0.538576798404693, 1.28..."
28892,3.0,91.0,Scrambled,sub_14,2.0,"[0.5014634340731977, -1.1395803584012405, 0.64..."
28893,4.0,91.0,Scrambled,sub_14,2.0,"[-0.9271863682797713, -0.549383417681263, 0.95..."
28894,5.0,91.0,Scrambled,sub_14,2.0,"[-0.8258939319598915, -0.17669852030324748, 0...."


In [39]:
cdf['tc_concatenated'].apply(lambda a: len(a)).values

array([624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624,
       624, 624, 624, 624, 624, 622, 622, 622, 622, 622, 622, 622, 622,
       622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622,
       622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622,
       622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622, 622,
       622, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623,
       623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623,
       623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 623, 62

In [20]:
df.loc[lambda d: (d['contrast']=='Scrambled')&(d['sub']=='sub_12')&(d['JU']==53)&(d['D_bins']==1)]

,contrast,sub,ses,task,run,muvi,start_TR,end_TR,dim4,ISC_flavour,JU,D_bins,n_bins,n_muvis,n_sub,tc_mean_unfolded
24456,Scrambled,sub_12,ses_01,1,1,S1ballonD.avi,4,16,220,M_OR_S,53,1,6,36,9,"[-0.9094634584431192, 0.15901222293568346, 0.5..."
24504,Scrambled,sub_12,ses_01,1,1,S3breakfastD.avi,19,38,220,M_OR_S,53,1,6,36,9,"[-0.4784282430890459, -0.2523732099684264, 0.6..."
24552,Scrambled,sub_12,ses_01,1,1,S6cigaretteD.avi,56,73,220,M_OR_S,53,1,6,36,9,"[-1.760896663238522, -0.7293326599048299, -0.3..."
24600,Scrambled,sub_12,ses_01,1,1,S7flowersD.avi,98,117,220,M_OR_S,53,1,6,36,9,"[-1.5905371205873704, -0.3700287303665868, 0.1..."
24648,Scrambled,sub_12,ses_01,1,1,S8frameD.avi,140,167,220,M_OR_S,53,1,6,36,9,"[-2.2134875429333154, 1.1498413738506765, -0.6..."
24936,Scrambled,sub_12,ses_01,1,2,S8frameD.avi,4,31,220,M_OR_S,53,1,6,36,9,"[-0.3996842134360508, 0.25322848490660965, -0...."
24984,Scrambled,sub_12,ses_01,1,2,S6cigaretteD.avi,34,51,220,M_OR_S,53,1,6,36,9,"[-1.3029395236548895, -1.0725843973096136, 1.0..."
25032,Scrambled,sub_12,ses_01,1,2,S3breakfastD.avi,84,103,220,M_OR_S,53,1,6,36,9,"[-0.5366382493082061, 1.1599194624165288, 0.83..."
25080,Scrambled,sub_12,ses_01,1,2,S1ballonD.avi,126,138,220,M_OR_S,53,1,6,36,9,"[-1.117850120333843, 1.2117132385970888, 1.495..."
25128,Scrambled,sub_12,ses_01,1,2,S7flowersD.avi,163,182,220,M_OR_S,53,1,6,36,9,"[-0.31085060798686215, -0.9435196532362783, 0...."


In [118]:
movie_names[c]

array(['M12letterD.avi', 'M13lightD.avi', 'M14nailD.avi', 'M15juiceD.avi',
       'M16pencilD.avi', 'M17pillowD.avi', 'M18removerD.avi',
       'M19sandwichD.avi', 'M1ballonD.avi', 'M21toastD.avi',
       'M22shirtD.avi', 'M2boatD.avi', 'M3breakfastD.avi',
       'M4buttonD.avi', 'M5cardD.avi', 'M6cigaretteD.avi',
       'M7flowersD.avi', 'M8frameD.avi'], dtype=object)

In [119]:
movie_list

array(['M13lightD.avi', 'M16pencilD.avi', 'M18removerD.avi',
       'M19sandwichD.avi', 'M22shirtD.avi', 'M2boatD.avi',
       'M4buttonD.avi', 'M5cardD.avi'], dtype=object)

In [76]:
concatenated

array([-0.10063118,  1.03533739,  0.28153692, -0.03453639,  1.31436405,
        0.17037449, -1.89025648, -1.68539773,  0.81093875,  0.09827017,
        0.38096231,  0.82067976,  0.2656863 , -2.37619736, -0.78376192,
       -0.6058662 , -0.33881884,  0.88065969,  1.03614526,  0.72051099,
        1.79451722, -0.62132233,  0.22196   ,  0.60343368,  0.24925965,
        0.11526566, -2.48202401,  0.09177084, -0.38265258,  1.12212246,
       -0.16033726, -0.55199332,  0.12320743,  0.44511459,  1.9338236 ,
       -0.44680525, -0.57336633, -1.48398288, -0.2320584 ,  1.89207467,
       -0.26014918, -1.14195165,  0.24497913, -0.50088574, -0.0452747 ,
        0.27875249,  2.60916573,  0.11399514, -1.08444325,  0.03749365,
       -0.63488371, -1.1603819 ,  0.65106098,  0.09457136, -0.8600558 ,
        0.49581068,  0.78730747,  2.37443858, -1.01156661, -0.09051985,
       -1.02316495, -1.31431737,  0.65436298, -0.24770316, -0.25772339,
       -0.36692438, -0.1110062 ,  1.72663434, -0.35436764,  0.64

In [78]:
gdf[['contrast','sub','ses','task','JU','D_bins']]

,contrast,sub,ses,task,JU,D_bins
62407,Scrambled,sub_14,ses_01,2,91,10
63707,Scrambled,sub_14,ses_01,2,91,10
64407,Scrambled,sub_14,ses_02,3,91,10
65797,Scrambled,sub_14,ses_02,3,91,10
62507,Scrambled,sub_14,ses_01,2,91,10
63607,Scrambled,sub_14,ses_01,2,91,10
62607,Scrambled,sub_14,ses_01,2,91,10
63507,Scrambled,sub_14,ses_01,2,91,10
64507,Scrambled,sub_14,ses_02,3,91,10
65687,Scrambled,sub_14,ses_02,3,91,10
